## Loading dataset

Import libraries needed in this notebook

In [ ]:
import pandas as pd 
import numpy as np 
%matplotlib inline
import matplotlib as mp 
import matplotlib.pyplot as plt
import seaborn as sns


here are the first five rows

In [ ]:
data_raw = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
data_raw.head()

dataset attributes and their data types

In [ ]:
data_raw.info()

In [ ]:
data_raw.dropna(inplace=True)

Number of sincere and insincere questions in this dataset

In [ ]:
data_raw.target.value_counts()

Let's plot their counts

In [ ]:
sns.countplot(data=data_raw, x='target')

In [ ]:
val_counts = data_raw.target.value_counts()
sincere_q_pc = val_counts[0]/val_counts.sum()
sincere_q_pc = sincere_q_pc*100
print('{}% of questions are sincere while the rest are insincere'.format(sincere_q_pc))

The vast majority of questions in this dataset are sincere and only a small number are insincere 

Let us take a look at the questions in this dataset and try finding out how they have been classified as sincere and insincere.

In [ ]:
insincere_questions = data_raw[data_raw['target'] == 1].question_text
sincere_questions = data_raw[data_raw['target'] == 0].question_text

In [ ]:
insincere_questions.sample(3, random_state=1).values

Sweet lord, do these questions look pathetic!

These are examples of some of the sincere questions

In [ ]:
sincere_questions.sample(3, random_state=1).values

## EDA and Visualization

- Before we start analysing text data, it is important that it is filtered of stop words, punctuations, contractions, and every document of text is tokenized.

- After cleaning up and tokenizing text, we need to convert every word to its root form so analysis and querying of data can be sped up.

### Cleaning of text data and preprocessing

Importing libraries for text clean up

In [ ]:
import nltk
import sys
import spacy

#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')

from nltk.corpus import stopwords

import string

Function for cleaning text

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=['parser','ner'])
stop = set(stopwords.words('english'))
punc = set(string.punctuation)

def clean_text(text):
    # Convert the text into lowercase
    text = text.lower()
    # Split into list
    wordList = text.split()
    # Remove punctuation
    wordList = ["".join(x for x in word if (x=="'")|(x not in punc)) for word in wordList]
    # Remove stop words
    wordList = [word for word in wordList if word not in stop]

    reformed_sentence = " ".join(wordList)
    doc = nlp(reformed_sentence)
    return " ".join([token.lemma_ for token in doc])

Let us take a sample question and run this function on it to see if it works

In [ ]:
question = data_raw.question_text.sample(1, random_state=1).values[0]
question

In [ ]:
clean_text(question)

Nice. Our text clean up function works

Now we will clean every row of text data by running this function

In [ ]:
data_raw['clean_text'] = data_raw['question_text'].astype('str').apply(clean_text)

### Visualizations

Build word clouds of sincere and insincere questions to find the most frequently occuring words in each.

Import the wordcloud library

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator

function for splitting sentences into a dictionary of uniquely occuring words and their frequencies

In [ ]:
def word_freq_dict(text):
    # Convert text into word list
    wordList = text.split()
    # Generate word freq dictionary
    wordFreqDict = {word: wordList.count(word) for word in wordList}
    return wordFreqDict

function for plotting a wordcloud from a word frequency dictionary

In [ ]:
def word_cloud_from_frequency(word_freq_dict, title, figure_size=(10,6)):
    wordcloud.generate_from_frequencies(word_freq_dict)
    plt.figure(figsize=figure_size)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title(title)
    plt.show()

Wordcloud of a random sample of 1000 insincere questions

In [ ]:
# join every question as a single sample of text
insincere_questions = data_raw.clean_text[data_raw['target'] == 1]
insincere_sample = " ".join(insincere_questions.sample(1000, random_state=1).values)
insincere_word_freq = word_freq_dict(insincere_sample)
wordcloud = WordCloud(width= 5000,
    height=3000,
    max_words=200,
    colormap='Reds',
    background_color='white')

word_cloud_from_frequency(insincere_word_freq, "Most Frequent Words in a sample of 1000 questions flagged insincere") 

Wordcloud of a random sample of sincere questions

In [ ]:
sincere_questions = data_raw[data_raw['target'] == 0].clean_text
sincere_sample = " ".join(sincere_questions.sample(1000, random_state=1).astype('str').values)
sincere_word_freq = word_freq_dict(sincere_sample)
wordcloud = WordCloud(width= 5000,
    height=3000,
    max_words=200,
    colormap='Greens',
    background_color='white')

word_cloud_from_frequency(sincere_word_freq, "Most Frequent Words in a sample of 1000 questions flagged sincere") 

## Text to vector matrix conversion 

We need to transform text into a matrix of vectors

### Using Bag of words model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_converter = CountVectorizer()

In [ ]:
sample_question_text = data_raw['clean_text'].sample(1, random_state= 1).values
sample_question_text

In [ ]:
sample_count_vectorized_data = bow_converter.fit_transform(sample_question_text)
sample_count_vectorized_data.toarray()

In [ ]:
count_vectorized_data_feature_names = bow_converter.get_feature_names()
count_vectorized_data_feature_names

In [ ]:
len(count_vectorized_data_feature_names)

### Using Tfidf(term-frequency-inverse-document-frequency) model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_converter = TfidfVectorizer(ngram_range=(1,1))

In [ ]:
sample_tfidf_vectorized_data = tfidf_converter.fit_transform(sample_question_text)
sample_tfidf_vectorized_data.toarray()

In [ ]:
tfidf_word_feature_names = tfidf_converter.get_feature_names()

In [ ]:
tfidf_word_feature_names

In [ ]:
len(tfidf_word_feature_names)

## Building a logistic regression model

### Pipeline with logistic regression and count vectorizer

define count vactorizer and logistic regression models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

count_vectorizer = CountVectorizer()
model = LogisticRegression(C=1, random_state=0)

vectorize_logit_pipeline = Pipeline([
    ('count_vectorizer', count_vectorizer),
    ('logit', model)
])

- define input and target variables
- split training dataset into train and test sets
- train the model using the feature and target training sets

In [ ]:
# input variable
X = data_raw['clean_text']
# target variable
y = data_raw['target']

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3)
vectorize_logit_pipeline.fit(train_X, train_y)

Get the predictions from the model

In [ ]:
predictions = vectorize_logit_pipeline.predict(test_X)

Check the accuracy score

In [ ]:
accuracy_score(test_y, predictions)

Check the f1 score and plot the confusion matrix

In [ ]:
from sklearn.metrics import f1_score
f1_score(test_y, predictions)

In [ ]:
confusion_matrix_logit_cv = confusion_matrix(test_y, predictions)
sns.heatmap(confusion_matrix_logit_cv, annot= True, xticklabels=['sincere', 'insincere'], yticklabels=['sincere', 'insincere'])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y, predictions))

### Define a pipeline with logistic regression and tfidf bi-grams vectorizer

In [ ]:
tfidf_ngrams_converter = TfidfVectorizer(ngram_range=(1,2))
tfidf_ngrams_logit_pipeline = Pipeline([
    ('tfidf_vectorizer', tfidf_ngrams_converter),
    ('logit', model)
])

In [ ]:
tfidf_ngrams_logit_pipeline.fit(train_X, train_y)

In [ ]:
new_predictions = tfidf_ngrams_logit_pipeline.predict(test_X)

In [ ]:
accuracy_score(test_y, new_predictions)

In [ ]:
f1_score(test_y, new_predictions)

In [ ]:
confusion_matrix_logit_tfidf = confusion_matrix(test_y, new_predictions)
sns.heatmap(confusion_matrix_logit_tfidf, annot= True, xticklabels=['sincere', 'insincere'], yticklabels=['sincere', 'insincere'])

In [ ]:
print(classification_report(test_y, new_predictions, target_names=['sincere', 'insincere']))

we can observe that the when bigram word vector features are included, our logit model gave us the best accuracy and f1 scores.

In [ ]:
test_data = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
test_data.head()

In [ ]:
test_data.info()

In [ ]:
test_data['clean_text'] = test_data['question_text'].astype('str').apply(clean_text)

In [ ]:
test_data.head()

In [ ]:
X_final = test_data['clean_text']

In [ ]:
y_final = tfidf_ngrams_logit_pipeline.predict(X_final)

In [ ]:
y_final[:5]

In [ ]:
test_data['target'] = y_final

In [ ]:
result_df = test_data[['qid', 'target']]

In [ ]:
result_df.rename(columns={'target': 'prediction'}, inplace=True)
result_df.set_index('qid', inplace=True)
result_df.head()

In [ ]:
result_df.to_csv('submission.csv')
!head submission.csv